In [46]:
import os 

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

In [47]:
pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Choice
chosenAlternative = travel_mode

In [48]:
#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)
CarOwn = Beta('CarOwn',0,None,None,0)


# socio-economic factors (interacting with Time)
Time_Female = Beta('Time_Female', 0, None, None, 0)
Time_Age = Beta('Time_Age', 0, None, None, 0)

In [49]:
# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit, database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)

##### Model with Gender as the socio-econ interaction

In [50]:
# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking + Time_Female * dur_walking * female
Opt2 = Constant2 + Total_TT2 * dur_cycling + Time_Female * dur_cycling * female
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + Time_Female * dur_public * female
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + CarOwn * car_ownership + \
        Time_Female * dur_driving * female
    
V = {1: Opt1,2: Opt2,3: Opt3, 4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

In [51]:
output_dir = "./model2-output"
filepath = os.path.join(output_dir, "logit_lpmc_model2_CarOwn+Gender")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
if os.path.exists(filepath):
    os.remove(f"{filepath}.html")
    
# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults

Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -3786.569
rho bar square = 0.452
Output file: ./model2-output/logit_lpmc_model2_CarOwn+Gender~04.html


,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
CarOwn,1.48,0.0533,27.9,0,0.0516,28.7,0
Constant2,-4.64,0.175,-26.5,0,0.224,-20.7,0
Constant3,-2.36,0.119,-19.9,0,0.161,-14.7,0
Constant4,-3.46,0.128,-26.9,0,0.173,-20,0
Cost,-0.159,0.0163,-9.75,0,0.0166,-9.6,0
Time_Female,-0.0677,0.251,-0.27,0.787,0.281,-0.241,0.809
Total_TT1,-8.17,0.331,-24.7,0,0.551,-14.8,0
Total_TT2,-5.64,0.469,-12,0,0.546,-10.3,0
Total_TT3,-3.3,0.276,-11.9,0,0.291,-11.3,0
Total_TT4,-6.13,0.368,-16.6,0,0.416,-14.7,0


##### Model with Age as the socio-econ interaction

In [52]:
# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking + Time_Age * dur_walking * age
Opt2 = Constant2 + Total_TT2 * dur_cycling + Time_Age * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + Time_Age * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + CarOwn * car_ownership +\
                    Time_Age * dur_driving * age
    
V = {1: Opt1,2: Opt2,3: Opt3, 4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

In [53]:
output_dir = "./model2-output"
filepath = os.path.join(output_dir, "logit_lpmc_model2_CarOwn+Age")

if not os.path.exists(output_dir):
    os.mkdir(output_dir)
if os.path.exists(filepath):
    os.remove(f"{filepath}.html")
    
# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults

Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -3763.172
rho bar square = 0.456
Output file: ./model2-output/logit_lpmc_model2_CarOwn+Age~02.html


,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
CarOwn,1.47,0.0534,27.6,0,0.0519,28.4,0
Constant2,-4.67,0.175,-26.6,0,0.222,-21,0
Constant3,-2.38,0.118,-20.1,0,0.157,-15.1,0
Constant4,-3.47,0.128,-27.1,0,0.171,-20.3,0
Cost,-0.161,0.0165,-9.77,0,0.017,-9.48,0
Time_Age,-0.0451,0.00665,-6.78,1.2e-11,0.00697,-6.47,9.71e-11
Total_TT1,-6.57,0.375,-17.5,0,0.559,-11.7,0
Total_TT2,-3.98,0.511,-7.79,6.66e-15,0.579,-6.87,6.37e-12
Total_TT3,-1.65,0.343,-4.8,1.56e-06,0.354,-4.66,3.22e-06
Total_TT4,-4.48,0.417,-10.7,0,0.457,-9.8,0
